In [1]:
from libpysal.weights import *
import numpy as np
import xarray as xr
%reload_ext memory_profiler

/data/GSoC/libpysal/libpysal/weights/util.py:20: UserWarning: geopandas not available. Some functionality will be disabled.
  warn('geopandas not available. Some functionality will be disabled.')


In [2]:
da = xr.open_rasterio("nasadem_sd.tif")
da.shape

(1, 3515, 5510)

*Profiling sparse weight builders for raster data*

`lat2SW` is very performant since it creates regular lattice without any missing values to deal with. It straight away creates diagonals and offsets which are then shipped to dia matrix constructor.
Memory consumpttion is high due to use of `list` instead of `np.array`

In [3]:
# memory profiling vanilla lat2SW
%memit lat2SW(*da[0].shape, "queen")

peak memory: 4093.32 MiB, increment: 3860.39 MiB


In [4]:
# perf-testing vanilla lat2SW
%timeit lat2SW(*da[0].shape, "queen")

6.22 s ± 452 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Next is `da2WSP` which uses `lat2SW` to build sparse matrix and then through boolean indexing it removes missing rows and columns from the created matrix.
Performance takes a hit due to boolean indexing of csr matrix.

In [5]:
%memit da2WSP(da, "queen")

peak memory: 4092.77 MiB, increment: 3859.39 MiB


In [6]:
%%timeit
da2WSP(da, "queen")

7.86 s ± 173 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To create matrix with only non missing values in mind, `lat2SW`/`dia_matrix` was out of question since it is higly oriented towards building regular lattice.

Next options were to build either [DOK](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html#scipy.sparse.dok_matrix), [CSR/CSC](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), or [COO](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix)...

- It was impossible (for me) to Numba-fy `DOK` builder due to its structure and without numba it was too slow.
- In case of `CSR/CSC` I was not able to think of a multithreaded implementation (still exploring a way to build this)
- From the start I was biased towards `COO` as its structure is quite simple, can be numba-fied, farely easy to incorporate multithreading and fast conversion to `CSR/CSC` matrix

In [7]:
# memory profiling COO based da2WSP
%memit da2WSPn(da)

peak memory: 2452.08 MiB, increment: 2145.65 MiB


In [8]:
# memory profiling
from libpysal.weights.raster import da2WSPn
%mprun -T da2WSPn_mem -f da2WSPn da2WSPn(da, "queen")



*** Profile printout saved to text file da2WSPn_mem. 


In [9]:
txt = open("da2WSPn_mem","r")
print(txt.read())

Filename: /data/GSoC/libpysal/libpysal/weights/raster.py

Line #    Mem usage    Increment   Line Contents
   348    328.8 MiB    328.8 MiB   def da2WSPn(da, criterion="queen", layer=None, dims=None):
   349    328.8 MiB      0.0 MiB       layer_id = _da_checker(da, layer, dims)[0]
   350    328.8 MiB      0.0 MiB       shape = da.shape
   351    328.8 MiB      0.0 MiB       if layer_id:
   352    328.8 MiB      0.0 MiB           da = da[layer_id-1:layer_id]
   353    328.8 MiB      0.0 MiB           shape = da[0].shape
   354    439.5 MiB    110.8 MiB       ser = da.to_series()
   355    439.5 MiB      0.0 MiB       mask = (ser != da.nodatavals[0]).to_numpy()
   356    551.2 MiB    111.7 MiB       ids = np.where(mask)[0]
   357    551.2 MiB      0.0 MiB       dtype = np.uint32 if (shape[0] * shape[1]) < 65535**2 else np.uint64
   358    551.2 MiB      0.0 MiB       n = len(ids)
   359   1166.5 MiB      0.0 MiB       sw = sparse.coo_matrix(
   360   1557.6 MiB   1006.4 MiB           _s

In [10]:
# perf-testing COO based da2WSP
%timeit da2WSPn(da, "queen")

4.46 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# without numba :)
%timeit da2WSPnn(da, "queen")

2min 7s ± 5.05 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#single threaded version
def da2WSP2(da, criterion="queen", layer=None, dims=None):
    layer_id = _da_checker(da, layer, dims)[0]
    shape = da.shape
    if layer_id:
        da = da[layer_id-1:layer_id]
        shape = da[0].shape
    ser = da.to_series()
    mask = (ser != da.nodatavals[0]).to_numpy()
    ids = np.where(mask)[0]
    dtype = np.uint32 if (shape[0] * shape[1]) < 65535**2 else np.uint64
    n = len(ids)
    sw = sparse.coo_matrix(
        _swbuilder(*shape, ids, mask, criterion, dtype), 
        shape=(n, n), 
        dtype=np.int8
    ).tocsr()
    ser = ser[ser != da.nodatavals[0]]
    index = ser.index
    wsp = WSP(sw, index=index)
    return wsp


@njit(fastmath=True)
def _swbuilder(nrows, ncols, ids, mask, criterion, dtype):
    n = len(ids)
    d = 4 if criterion == "rook" else 8
    # id map
    id_map = mask * 1
    id_map[ids] = np.arange(len(ids), dtype=dtype)
    # preallocating rows and cols 
    rows = np.empty(d*n, dtype=dtype)
    cols = np.empty(d*n, dtype=dtype)
    ni = 0
    for i in range(n):
        id_i = ids[i]
        if ((id_i+1) % ncols) != 0:
            og_id = id_map[id_i]
            id_neighbor = id_map[id_i + 1]
            if id_neighbor:
                rows[ni], cols[ni] = og_id, id_neighbor
                ni += 1
                rows[ni], cols[ni] = id_neighbor, og_id
                ni += 1
        if (id_i // ncols) < (nrows - 1):
            og_id = id_map[id_i]
            id_neighbor = id_map[id_i+ncols]
            if id_neighbor:
                rows[ni], cols[ni] = og_id, id_neighbor
                ni += 1
                rows[ni], cols[ni] = id_neighbor, og_id
                ni += 1
        if criterion == "queen":
            if (id_i // ncols) < (nrows - 1):
                if (id_i % ncols) != 0:
                    og_id = id_map[id_i]
                    id_neighbor = id_map[id_i+ncols-1]
                    if id_neighbor:
                        rows[ni], cols[ni] = og_id, id_neighbor
                        ni += 1
                        rows[ni], cols[ni] = id_neighbor, og_id
                        ni += 1
                if ((id_i+1) % ncols) != 0:
                    og_id = id_map[id_i]
                    id_neighbor = id_map[id_i+ncols+1]
                    if id_neighbor:
                        rows[ni], cols[ni] = og_id, id_neighbor
                        ni += 1
                        rows[ni], cols[ni] = id_neighbor, og_id
                        ni += 1
    rows = rows[:ni].copy()
    cols = cols[:ni].copy()
    data = np.ones(ni, dtype=np.int8)
    return (data, (rows, cols))

In [ ]:
# WIP parallel implementation
@njit(fastmath=True)
def compute_chunk(nrows, ncols, ids, id_map, criterion, dtype):
    n = len(ids)
    d = 4 if criterion == "rook" else 8
    # id map
    rows = np.empty(d*n, dtype=dtype)
    cols = np.empty(d*n, dtype=dtype)
    ni = 0
    for i in range(n):
        id_i = ids[i]
        if ((id_i+1) % ncols) != 0:
            og_id = id_map[id_i]
            id_neighbor = id_map[id_i + 1]
            if id_neighbor:
                rows[ni], cols[ni] = og_id, id_neighbor
                ni += 1
                rows[ni], cols[ni] = id_neighbor, og_id
                ni += 1
        if (id_i // ncols) < (nrows - 1):
            og_id = id_map[id_i]
            id_neighbor = id_map[id_i+ncols]
            if id_neighbor:
                rows[ni], cols[ni] = og_id, id_neighbor
                ni += 1
                rows[ni], cols[ni] = id_neighbor, og_id
                ni += 1
        if criterion == "queen":
            if (id_i // ncols) < (nrows - 1):
                if (id_i % ncols) != 0:
                    og_id = id_map[id_i]
                    id_neighbor = id_map[id_i+ncols-1]
                    if id_neighbor:
                        rows[ni], cols[ni] = og_id, id_neighbor
                        ni += 1
                        rows[ni], cols[ni] = id_neighbor, og_id
                        ni += 1
                if ((id_i+1) % ncols) != 0:
                    og_id = id_map[id_i]
                    id_neighbor = id_map[id_i+ncols+1]
                    if id_neighbor:
                        rows[ni], cols[ni] = og_id, id_neighbor
                        ni += 1
                        rows[ni], cols[ni] = id_neighbor, og_id
                        ni += 1
    rows = rows[:ni].copy()
    cols = cols[:ni].copy()
    return rows, cols, ni


@njit(fastmath=True)
def chunk_generator(n_jobs, starts, ids):
    chunk_size = starts[1] - starts[0]
    for i in range(n_jobs):
        start = starts[i]
        ids_chunk = ids[start: (start + chunk_size)]
        yield (ids_chunk,)


@njit(fastmath=True)
def _idmap(ids, mask, dtype):
    mask = mask * 1
    mask[ids] = np.arange(len(ids), dtype=dtype)
    return mask


def _swbuilder(nrows, ncols, ids, mask, criterion, dtype):
    from joblib import Parallel, delayed
    n = len(ids)
    n_jobs = os.cpu_count()
    chunk_size = n // n_jobs + 1
    starts = np.arange(n_jobs + 1) * chunk_size
    id_map = _idmap(ids, mask, dtype)
    chunk = chunk_generator(n_jobs, starts, ids)
    worker_out = Parallel(n_jobs=-1)(
        delayed(compute_chunk)(nrows, ncols, *pars, id_map, criterion, dtype)
        for pars in chunk
    )
    rows, cols, j = zip(*worker_out)
    rows = np.hstack(rows).flatten()
    cols = np.hstack(cols).flatten()
    data = np.ones(np.sum(j), dtype=np.int8)
    return (data, (rows, cols))
